# Bernoulli Console With Random Orientation

In [152]:
L, d, t = 1000.0, 100.0, 5.0
Ex, nu = 210000.0, 0.25
penalty = 1e20  # penalty value


In [153]:
seed = 0  # integer or None
seed = None


## Linear Solution

In [154]:
from linkeddeepdict import LinkedDeepDict
from linkeddeepdict.tools import getallfromkwargs
from neumann.linalg import Vector, linspace
from neumann.array import repeat
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from polymesh.space.utils import index_of_closest_point, index_of_furthest_point
from polymesh.topo.tr import L2_to_L3
from sigmaepsilon.solid.fem.cells import B2, B3
from sigmaepsilon.solid import Structure, LineMesh, PointData, BeamSection
import numpy as np
from latexdocs.utils import floatformatter

rs = np.random.RandomState()
if isinstance(seed, int):
    rs.seed(seed)

formatter = floatformatter(sig=6)
def f2s(x): return formatter.format(x)


# section
section = BeamSection('CHS', d=d, t=t, n=32)
section.calculate_section_properties()
section_props = section.section_properties
A, Ix, Iy, Iz = getallfromkwargs(['A', 'Ix', 'Iy', 'Iz'], **section_props)

# material
G = Ex / (2 * (1 + nu))
Hooke = np.array([
    [Ex*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, Ex*Iy, 0],
    [0, 0, 0, Ex*Iz]
])


def solve(n, angles, loads, celltype=B2):
    # space
    GlobalFrame = StandardFrame(dim=3)
    TargetFrame = GlobalFrame.rotate('Body', angles, 'XYZ', inplace=False)

    # mesh
    p0 = np.array([0., 0., 0.])
    p1 = np.array([L, 0., 0.])
    coords = linspace(p0, p1, n+1)
    points = PointCloud(coords, frame=TargetFrame)
    coords = points.show(GlobalFrame)
    topo = np.zeros((n, 2), dtype=int)
    topo[:, 0] = np.arange(n)
    topo[:, 1] = np.arange(n) + 1
    if celltype.NNODE == 3:
        coords, topo = L2_to_L3(coords, topo)
    i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
    i_last = index_of_furthest_point(coords, np.array([0., 0., 0.]))

    # essential boundary conditions
    fixity = np.zeros((coords.shape[0], 6)).astype(bool)
    fixity[i_first, :] = True
    fixity = fixity.astype(float) * penalty

    # natural boundary conditions
    loads = np.array(loads)
    nodal_loads = np.zeros((coords.shape[0], 6))
    vF = Vector(loads[:3], frame=TargetFrame).show(GlobalFrame)
    vM = Vector(loads[3:], frame=TargetFrame).show(GlobalFrame)
    nodal_loads[i_last, :3] = vF
    nodal_loads[i_last, 3:] = vM

    # pointdata
    pd = PointData(coords=coords, frame=GlobalFrame,
                   loads=nodal_loads, fixity=fixity)

    # celldata
    frames = repeat(TargetFrame.axes, topo.shape[0])
    cd = celltype(topo=topo, frames=frames)

    # set up mesh and structure
    mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
    structure = Structure(mesh=mesh)

    structure.linsolve()

    dofsol = structure.nodal_dof_solution(store='dofsol')
    u = np.zeros(6)
    u[:3] = Vector(dofsol[i_last, :3], frame=GlobalFrame).show(TargetFrame)
    u[3:] = Vector(dofsol[i_last, 3:], frame=GlobalFrame).show(TargetFrame)

    reactions = structure.reaction_forces()
    r = np.zeros(6)
    r[:3] = Vector(reactions[i_first, :3], frame=GlobalFrame).show(TargetFrame)
    r[3:] = Vector(reactions[i_first, 3:], frame=GlobalFrame).show(TargetFrame)

    forces = structure.internal_forces()
    f = np.zeros(6)
    f[:3] = forces[0, 0, :3]
    f[3:] = forces[0, 0, 3:]

    return u, r, f


## Tests

In [155]:
def absolute_relative_error(v_analytic, v_fem):
    return np.abs(100 * (v_analytic - v_fem) / v_analytic)

### Concentrated forces at the free end

In [156]:
n = 2  # number of elements

Concentrated force in local X

In [157]:
Fx = 1.0
loads = [Fx, 0, 0, 0, 0, 0]

# analytical solutions
UX = Fx * L / (Ex * A)  # displacement at the free end
RX = -Fx  # reaction force
FX = Fx  # internal force at the support

In [158]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B2)
    UX_fem = u[0]
    RX_fem = r[0]
    FX_fem = f[0]
    print("UX  | Analytic : {}, FEM : {}".format(f2s(UX), f2s(UX_fem)))
    print("RX  | Analytic : {}, FEM : {}".format(f2s(RX), f2s(RX_fem)))
    print("FX  | Analytic : {}, FEM : {}".format(f2s(FX), f2s(FX_fem)))
    print("\n")


UX  | Analytic : 3.21167e-06, FEM : 3.21167e-06
RX  | Analytic : -1, FEM : -1
FX  | Analytic : 1, FEM : 1


UX  | Analytic : 3.21167e-06, FEM : 3.21167e-06
RX  | Analytic : -1, FEM : -1
FX  | Analytic : 1, FEM : 1


UX  | Analytic : 3.21167e-06, FEM : 3.21167e-06
RX  | Analytic : -1, FEM : -1
FX  | Analytic : 1, FEM : 1




In [159]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B3)
    UX_fem = u[0]
    RX_fem = r[0]
    FX_fem = f[0]
    print("UX  | Analytic : {}, FEM : {}".format(f2s(UX), f2s(UX_fem)))
    print("RX  | Analytic : {}, FEM : {}".format(f2s(RX), f2s(RX_fem)))
    print("FX  | Analytic : {}, FEM : {}".format(f2s(FX), f2s(FX_fem)))
    print("\n")

UX  | Analytic : 3.21167e-06, FEM : 3.21167e-06
RX  | Analytic : -1, FEM : -1
FX  | Analytic : 1, FEM : 1


UX  | Analytic : 3.21167e-06, FEM : 3.21167e-06
RX  | Analytic : -1, FEM : -1
FX  | Analytic : 1, FEM : 1


UX  | Analytic : 3.21167e-06, FEM : 3.21167e-06
RX  | Analytic : -1, FEM : -1
FX  | Analytic : 1, FEM : 1




Contentrated force in local Y

In [160]:
Fy = 1.0
loads = [0, Fy, 0, 0, 0, 0]

# analytical solutions
UY = Fy * L**3 / (3 * Ex * Iz)  # displacement at the free end
UZZ = Fy * L**2 / (2 * Ex * Iz)  # rotation at the free end
RY = -Fy  # reaction force
RZZ = -Fy * L  # reaction moment
FY = Fy  # internal force at the support
FZZ = Fy * L  # internal moment at the support

In [161]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B2)
    UY_fem, UZZ_fem = u[1], u[5]
    RY_fem, RZZ_fem = r[1], r[5]
    FY_fem, FZZ_fem = f[1], f[5]
    print("UY  | Analytic : {}, FEM : {}".format(f2s(UY), f2s(UY_fem)))
    print("UZZ | Analytic : {}, FEM : {}".format(f2s(UZZ), f2s(UZZ_fem)))
    print("RY  | Analytic : {}, FEM : {}".format(f2s(RY), f2s(RY_fem)))
    print("RZZ | Analytic : {}, FEM : {}".format(f2s(RZZ), f2s(RZZ_fem)))
    print("FY  | Analytic : {}, FEM : {}".format(f2s(FY), f2s(FY_fem)))
    print("FZZ | Analytic : {}, FEM : {}".format(f2s(FZZ), f2s(FZZ_fem)))
    print("\n")

UY  | Analytic : 0.00095245, FEM : 0.00095245
UZZ | Analytic : 1.42867e-06, FEM : 1.42867e-06
RY  | Analytic : -1, FEM : -1
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 1, FEM : 1
FZZ | Analytic : 1000, FEM : 1000


UY  | Analytic : 0.00095245, FEM : 0.00095245
UZZ | Analytic : 1.42867e-06, FEM : 1.42867e-06
RY  | Analytic : -1, FEM : -1
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 1, FEM : 1
FZZ | Analytic : 1000, FEM : 1000


UY  | Analytic : 0.00095245, FEM : 0.00095245
UZZ | Analytic : 1.42867e-06, FEM : 1.42867e-06
RY  | Analytic : -1, FEM : -1
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 1, FEM : 1
FZZ | Analytic : 1000, FEM : 1000




In [162]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B3)
    UY_fem, UZZ_fem = u[1], u[5]
    RY_fem, RZZ_fem = r[1], r[5]
    FY_fem, FZZ_fem = f[1], f[5]
    print("UY  | Analytic : {}, FEM : {}".format(f2s(UY), f2s(UY_fem)))
    print("UZZ | Analytic : {}, FEM : {}".format(f2s(UZZ), f2s(UZZ_fem)))
    print("RY  | Analytic : {}, FEM : {}".format(f2s(RY), f2s(RY_fem)))
    print("RZZ | Analytic : {}, FEM : {}".format(f2s(RZZ), f2s(RZZ_fem)))
    print("FY  | Analytic : {}, FEM : {}".format(f2s(FY), f2s(FY_fem)))
    print("FZZ | Analytic : {}, FEM : {}".format(f2s(FZZ), f2s(FZZ_fem)))
    print("\n")

UY  | Analytic : 0.00095245, FEM : 0.00095245
UZZ | Analytic : 1.42867e-06, FEM : 1.42867e-06
RY  | Analytic : -1, FEM : -1
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 1, FEM : 1
FZZ | Analytic : 1000, FEM : 1000


UY  | Analytic : 0.00095245, FEM : 0.00095245
UZZ | Analytic : 1.42867e-06, FEM : 1.42867e-06
RY  | Analytic : -1, FEM : -1
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 1, FEM : 1
FZZ | Analytic : 1000, FEM : 1000


UY  | Analytic : 0.00095245, FEM : 0.00095245
UZZ | Analytic : 1.42867e-06, FEM : 1.42867e-06
RY  | Analytic : -1, FEM : -1
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 1, FEM : 1
FZZ | Analytic : 1000, FEM : 1000




Contentrated force in local Z

In [163]:
Fz = 1.0
loads = [0, 0, Fz, 0, 0, 0]

# analytical solutions
UZ = Fz * L**3 / (3 * Ex * Iy)  # displacement at the free end
UYY = -Fz * L**2 / (2 * Ex * Iy)  # rotation at the free end
RZ = -Fz  # reaction force
RYY = Fz * L  # reaction moment
FZ = Fz  # internal force at the support
FYY = -Fz * L  # internal moment at the support

In [164]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B2)
    UZ_fem, UYY_fem = u[2], u[4]
    RZ_fem, RYY_fem = r[2], r[4]
    FZ_fem, FYY_fem = f[2], f[4]
    print("UZ  | Analytic : {}, FEM : {}".format(f2s(UZ), f2s(UZ_fem)))
    print("UYY | Analytic : {}, FEM : {}".format(f2s(UYY), f2s(UYY_fem)))
    print("RZ  | Analytic : {}, FEM : {}".format(f2s(RZ), f2s(RZ_fem)))
    print("RYY | Analytic : {}, FEM : {}".format(f2s(RYY), f2s(RYY_fem)))
    print("FZ  | Analytic : {}, FEM : {}".format(f2s(FZ), f2s(FZ_fem)))
    print("FYY | Analytic : {}, FEM : {}".format(f2s(FYY), f2s(FYY_fem)))
    print("\n")


UZ  | Analytic : 0.00095245, FEM : 0.00095245
UYY | Analytic : -1.42867e-06, FEM : -1.42867e-06
RZ  | Analytic : -1, FEM : -1
RYY | Analytic : 1000, FEM : 1000
FZ  | Analytic : 1, FEM : 1
FYY | Analytic : -1000, FEM : -1000


UZ  | Analytic : 0.00095245, FEM : 0.00095245
UYY | Analytic : -1.42867e-06, FEM : -1.42867e-06
RZ  | Analytic : -1, FEM : -1
RYY | Analytic : 1000, FEM : 1000
FZ  | Analytic : 1, FEM : 1
FYY | Analytic : -1000, FEM : -1000


UZ  | Analytic : 0.00095245, FEM : 0.00095245
UYY | Analytic : -1.42867e-06, FEM : -1.42867e-06
RZ  | Analytic : -1, FEM : -1
RYY | Analytic : 1000, FEM : 1000
FZ  | Analytic : 1, FEM : 1
FYY | Analytic : -1000, FEM : -1000




In [165]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B3)
    UZ_fem, UYY_fem = u[2], u[4]
    RZ_fem, RYY_fem = r[2], r[4]
    FZ_fem, FYY_fem = f[2], f[4]
    print("UZ  | Analytic : {}, FEM : {}".format(f2s(UZ), f2s(UZ_fem)))
    print("UYY | Analytic : {}, FEM : {}".format(f2s(UYY), f2s(UYY_fem)))
    print("RZ  | Analytic : {}, FEM : {}".format(f2s(RZ), f2s(RZ_fem)))
    print("RYY | Analytic : {}, FEM : {}".format(f2s(RYY), f2s(RYY_fem)))
    print("FZ  | Analytic : {}, FEM : {}".format(f2s(FZ), f2s(FZ_fem)))
    print("FYY | Analytic : {}, FEM : {}".format(f2s(FYY), f2s(FYY_fem)))
    print("\n")


UZ  | Analytic : 0.00095245, FEM : 0.00095245
UYY | Analytic : -1.42867e-06, FEM : -1.42867e-06
RZ  | Analytic : -1, FEM : -1
RYY | Analytic : 1000, FEM : 1000
FZ  | Analytic : 1, FEM : 1
FYY | Analytic : -1000, FEM : -1000


UZ  | Analytic : 0.00095245, FEM : 0.00095245
UYY | Analytic : -1.42867e-06, FEM : -1.42867e-06
RZ  | Analytic : -1, FEM : -1
RYY | Analytic : 1000, FEM : 1000
FZ  | Analytic : 1, FEM : 1
FYY | Analytic : -1000, FEM : -1000


UZ  | Analytic : 0.00095245, FEM : 0.00095245
UYY | Analytic : -1.42867e-06, FEM : -1.42867e-06
RZ  | Analytic : -1, FEM : -1
RYY | Analytic : 1000, FEM : 1000
FZ  | Analytic : 1, FEM : 1
FYY | Analytic : -1000, FEM : -1000




### Concentrated moments at the free end

Moment around local X

In [166]:
Mx = 1.0
loads = [0, 0, 0, Mx, 0, 0]

# analytical solutions
UXX = Mx * L / (G * Ix)  # rotation at the free end
RXX = -Mx  # reaction moment
FXX = Mx  # internal moment at the support

In [167]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B2)
    UXX_fem = u[3]
    RXX_fem = r[3]
    FXX_fem = f[3]
    print("UXX | Analytic : {}, FEM : {}".format(f2s(UXX), f2s(UXX_fem)))
    print("RXX | Analytic : {}, FEM : {}".format(f2s(RXX), f2s(RXX_fem)))
    print("FXX | Analytic : {}, FEM : {}".format(f2s(FXX), f2s(FXX_fem)))
    print("\n")

UXX | Analytic : 3.57169e-09, FEM : 3.57169e-09
RXX | Analytic : -1, FEM : -1
FXX | Analytic : 1, FEM : 1


UXX | Analytic : 3.57169e-09, FEM : 3.57169e-09
RXX | Analytic : -1, FEM : -1
FXX | Analytic : 1, FEM : 1


UXX | Analytic : 3.57169e-09, FEM : 3.57169e-09
RXX | Analytic : -1, FEM : -1
FXX | Analytic : 1, FEM : 1




In [168]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B3)
    UXX_fem = u[3]
    RXX_fem = r[3]
    FXX_fem = f[3]
    print("UXX | Analytic : {}, FEM : {}".format(f2s(UXX), f2s(UXX_fem)))
    print("RXX | Analytic : {}, FEM : {}".format(f2s(RXX), f2s(RXX_fem)))
    print("FXX | Analytic : {}, FEM : {}".format(f2s(FXX), f2s(FXX_fem)))
    print("\n")

UXX | Analytic : 3.57169e-09, FEM : 3.57169e-09
RXX | Analytic : -1, FEM : -1
FXX | Analytic : 1, FEM : 1


UXX | Analytic : 3.57169e-09, FEM : 3.57169e-09
RXX | Analytic : -1, FEM : -1
FXX | Analytic : 1, FEM : 1


UXX | Analytic : 3.57169e-09, FEM : 3.57169e-09
RXX | Analytic : -1, FEM : -1
FXX | Analytic : 1, FEM : 1




Moment around local Y

In [169]:
My = 1.0
loads = [0, 0, 0, 0, My, 0]

# analytical solutions
UZ = -My * L**2 / (2 * Ex * Iy)  # displacement at the free end
UYY = My * L / (Ex * Iy)  # rotation at the free end
RZ = 0.  # reaction force
RYY = -My  # reaction moment
FZ = 0  # internal force at the support
FYY = My  # internal moment at the support

In [170]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B2)
    UZ_fem, UYY_fem = u[2], u[4]
    RZ_fem, RYY_fem = r[2], r[4]
    FZ_fem, FYY_fem = f[2], f[4]
    print("UZ  | Analytic : {}, FEM : {}".format(f2s(UZ), f2s(UZ_fem)))
    print("UYY | Analytic : {}, FEM : {}".format(f2s(UYY), f2s(UYY_fem)))
    print("RZ  | Analytic : {}, FEM : {}".format(f2s(RZ), f2s(RZ_fem)))
    print("RYY | Analytic : {}, FEM : {}".format(f2s(RYY), f2s(RYY_fem)))
    print("FZ  | Analytic : {}, FEM : {}".format(f2s(FZ), f2s(FZ_fem)))
    print("FYY | Analytic : {}, FEM : {}".format(f2s(FYY), f2s(FYY_fem)))
    print("\n")

UZ  | Analytic : -1.42867e-06, FEM : -1.42867e-06
UYY | Analytic : 2.85735e-09, FEM : 2.85735e-09
RZ  | Analytic : 0, FEM : 7.09544e-19
RYY | Analytic : -1, FEM : -1
FZ  | Analytic : 0, FEM : -1.06455e-18
FYY | Analytic : 1, FEM : 1


UZ  | Analytic : -1.42867e-06, FEM : -1.42867e-06
UYY | Analytic : 2.85735e-09, FEM : 2.85735e-09
RZ  | Analytic : 0, FEM : 3.34456e-17
RYY | Analytic : -1, FEM : -1
FZ  | Analytic : 0, FEM : -2.75684e-17
FYY | Analytic : 1, FEM : 1


UZ  | Analytic : -1.42867e-06, FEM : -1.42867e-06
UYY | Analytic : 2.85735e-09, FEM : 2.85735e-09
RZ  | Analytic : 0, FEM : 1.9128e-17
RYY | Analytic : -1, FEM : -1
FZ  | Analytic : 0, FEM : -2.31511e-17
FYY | Analytic : 1, FEM : 1




In [171]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B3)
    UZ_fem, UYY_fem = u[2], u[4]
    RZ_fem, RYY_fem = r[2], r[4]
    FZ_fem, FYY_fem = f[2], f[4]
    print("UZ  | Analytic : {}, FEM : {}".format(f2s(UZ), f2s(UZ_fem)))
    print("UYY | Analytic : {}, FEM : {}".format(f2s(UYY), f2s(UYY_fem)))
    print("RZ  | Analytic : {}, FEM : {}".format(f2s(RZ), f2s(RZ_fem)))
    print("RYY | Analytic : {}, FEM : {}".format(f2s(RYY), f2s(RYY_fem)))
    print("FZ  | Analytic : {}, FEM : {}".format(f2s(FZ), f2s(FZ_fem)))
    print("FYY | Analytic : {}, FEM : {}".format(f2s(FYY), f2s(FYY_fem)))
    print("\n")

UZ  | Analytic : -1.42867e-06, FEM : -1.42867e-06
UYY | Analytic : 2.85735e-09, FEM : 2.85735e-09
RZ  | Analytic : 0, FEM : 1.47625e-16
RYY | Analytic : -1, FEM : -1
FZ  | Analytic : 0, FEM : -1.33227e-15
FYY | Analytic : 1, FEM : 1


UZ  | Analytic : -1.42867e-06, FEM : -1.42867e-06
UYY | Analytic : 2.85735e-09, FEM : 2.85735e-09
RZ  | Analytic : 0, FEM : -3.00589e-17
RYY | Analytic : -1, FEM : -1
FZ  | Analytic : 0, FEM : -1.94289e-16
FYY | Analytic : 1, FEM : 1


UZ  | Analytic : -1.42867e-06, FEM : -1.42867e-06
UYY | Analytic : 2.85735e-09, FEM : 2.85735e-09
RZ  | Analytic : 0, FEM : 1.20208e-16
RYY | Analytic : -1, FEM : -1
FZ  | Analytic : 0, FEM : -1.66533e-16
FYY | Analytic : 1, FEM : 1




Moment around local Z

In [172]:
Mz = 1.0
loads = [0, 0, 0, 0, 0, Mz]

# analytical solutions
UY = Mz * L**2 / (2 * Ex * Iz)  # displacement at the free end
UZZ = Mz * L / (Ex * Iz)  # rotation at the free end
RY = 0.  # reaction force
RZZ = -Mz  # reaction moment
FY = 0.  # internal force at the support
FZZ = Mz  # internal moment at the support

In [173]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B2)
    UY_fem, UZZ_fem = u[1], u[5]
    RY_fem, RZZ_fem = r[1], r[5]
    FY_fem, FZZ_fem = f[1], f[5]
    print("UY  | Analytic : {}, FEM : {}".format(f2s(UY), f2s(UY_fem)))
    print("UZZ | Analytic : {}, FEM : {}".format(f2s(UZZ), f2s(UZZ_fem)))
    print("RY  | Analytic : {}, FEM : {}".format(f2s(RY), f2s(RY_fem)))
    print("RZZ | Analytic : {}, FEM : {}".format(f2s(RZZ), f2s(RZZ_fem)))
    print("FY  | Analytic : {}, FEM : {}".format(f2s(FY), f2s(FY_fem)))
    print("FZZ | Analytic : {}, FEM : {}".format(f2s(FZZ), f2s(FZZ_fem)))
    print("\n")

UY  | Analytic : 0.00142867, FEM : 0.00142867
UZZ | Analytic : 2.85735e-06, FEM : 2.85735e-06
RY  | Analytic : 0, FEM : 1.36659e-14
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 0, FEM : -1.17633e-14
FZZ | Analytic : 1000, FEM : 1000


UY  | Analytic : 0.00142867, FEM : 0.00142867
UZZ | Analytic : 2.85735e-06, FEM : 2.85735e-06
RY  | Analytic : 0, FEM : -5.7422e-14
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 0, FEM : 4.93015e-14
FZZ | Analytic : 1000, FEM : 1000


UY  | Analytic : 0.00142867, FEM : 0.00142867
UZZ | Analytic : 2.85735e-06, FEM : 2.85735e-06
RY  | Analytic : 0, FEM : -3.80936e-15
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 0, FEM : 1.53766e-14
FZZ | Analytic : 1000, FEM : 1000




In [174]:
for i in range(3):
    angles = np.random.rand(3) * np.pi * 2
    u, r, f = solve(n, angles, loads, B3)
    UY_fem, UZZ_fem = u[1], u[5]
    RY_fem, RZZ_fem = r[1], r[5]
    FY_fem, FZZ_fem = f[1], f[5]
    print("UY  | Analytic : {}, FEM : {}".format(f2s(UY), f2s(UY_fem)))
    print("UZZ | Analytic : {}, FEM : {}".format(f2s(UZZ), f2s(UZZ_fem)))
    print("RY  | Analytic : {}, FEM : {}".format(f2s(RY), f2s(RY_fem)))
    print("RZZ | Analytic : {}, FEM : {}".format(f2s(RZZ), f2s(RZZ_fem)))
    print("FY  | Analytic : {}, FEM : {}".format(f2s(FY), f2s(FY_fem)))
    print("FZZ | Analytic : {}, FEM : {}".format(f2s(FZZ), f2s(FZZ_fem)))
    print("\n")

UY  | Analytic : 0.00142867, FEM : 0.00142867
UZZ | Analytic : 2.85735e-06, FEM : 2.85735e-06
RY  | Analytic : 0, FEM : 6.5267e-13
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 0, FEM : -2.10321e-12
FZZ | Analytic : 1000, FEM : 1000


UY  | Analytic : 0.00142867, FEM : 0.00142867
UZZ | Analytic : 2.85735e-06, FEM : 2.85735e-06
RY  | Analytic : 0, FEM : 1.51802e-14
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 0, FEM : 5.68434e-14
FZZ | Analytic : 1000, FEM : 1000


UY  | Analytic : 0.00142867, FEM : 0.00142867
UZZ | Analytic : 2.85735e-06, FEM : 2.85735e-06
RY  | Analytic : 0, FEM : -6.6815e-14
RZZ | Analytic : -1000, FEM : -1000
FY  | Analytic : 0, FEM : 8.2423e-13
FZZ | Analytic : 1000, FEM : 1000


